## Create ELMo Vocabulary file


* Not required if you are doing finetuning


In [175]:
import nltk
nltk.download('punkt')

from nltk import word_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
def word_count_dict(PATH):

  ifile = open(PATH,'r')
  lines = ifile.readlines()

  wordlist = []
  countlist = []

  for line in lines:
      # Get all the words in the current line
      words = word_tokenize(line)

      words = line.split()
      for word in words:
          # Perform whatever manipulation to the word here
          # Remove any punctuation from the word
          # word = word.strip(".,!?=:();'\"")
          # Make the word lowercase
          word = word.lower()

          # Add the word into wordlist only if it is not in wordlist
          if word not in wordlist:
            wordlist.append(word)

          # Add the word to countlist so that it can be counted later
          countlist.append(word)

  # Sort the wordlist
  wordlist.sort()
  count_dict = dict()

  # Print the wordlist
  for word in wordlist:
      count_dict[word] = countlist.count(word)
    
  return count_dict

word_count = word_count_dict('/content/sample_data/training_data.txt')

In [0]:
sorted_word_list = [word[0] for word in sorted(word_count.items(), key=lambda item: item[1])][::-1]
custom_tokens =['<S>', '</S>','<UNK>']
custom_tokens.extend(sorted_word_list)

In [0]:
!>/content/elmo_vocab.txt

In [0]:
with open('/content/sample_data/elmo_vocab.txt', 'w') as f:
    for item in custom_tokens:
        f.write("%s\n" % item)

## Creating dataset

In [180]:
!mkdir /content/training_dataset

mkdir: cannot create directory ‘/content/training_dataset’: File exists


In [0]:
from random import shuffle

def divide_chunks(lst, n): 
  '''
  Divide the list into even chunks
  lst : list
  n: Number of required elements in each list
  '''      
    # looping till length l 
  return [lst[i:i + n] for i in range(0, len(lst), n)]

with open('/content/sample_data/training_data.txt','r') as txtfile:
  lines = txtfile.readlines()
sentences = [x.strip('.\n') for x in lines] 

# Remove empty strings from the list
cleaned_sentences = (list(filter(None, content)))

#Shuffle works in place
shuffle(cleaned_sentences)

chunk_list = divide_chunks(cleaned_sentences, 1415)


In [182]:
print(len(chunk_list))

10


### Create text files in the folder

In [0]:
import os

path = "/content/training_dataset/"

for i in range(0,9):

  file_path = path+"data_"+str(i)+'.txt'
  if not os.path.exists(file_path):
    os.mknod(file_path)

In [185]:
num = int(len(cleaned_sentences)/1415)
print(num)
for i in range(0,num):
  file_path = path+"data_"+str(i)+'.txt'

  with open(file_path, 'w') as txtfile:
    for item in cleaned_sentences[:num]:
        txtfile.write("%s\n" % item)
  num = num+num


9


## Cloning elmo files

In [0]:
# Downloading ELMo vocab
!wget https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/vocab-2016-09-10.txt /content/sample_data

In [0]:
! git clone https://github.com/allenai/bilm-tf /content/elmo_repo

### Finetuning data

In [0]:
!python /content/elmo_repo/bin/restart.py \
        --save_dir /content/sample_data/checkpoint\
        --vocab_file /content/vocab-2016-09-10.txt\
        --train_prefix='/content/training_dataset/*'\
        --n_epochs=5